# Scoring (future work)

# Do F1 score based on PDC challenges
location: Datasets/PDC19,PDC20,PDC21,PDC22
inside: Ground Truth Logs | Test Logs | Training Logs
exceptions: PDC20 have Models (pnml) and PDC22 has Base Logs